# Training & evaluation with the built-in methods
## API genel bakış: İlk end-to-end örnek 
Bir modele data gönderiyorsak, numpy array yada tf.data objesi kullanmamız gerekiyor. Data küçükse numpy array yeterli olabilir. 
Mnist datası için bir model oluşturalım.  

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Genellikle end-to-end bir akış aşağıdakileri içermelidir. 
* Training
* Validation (Training verisinden çıkarılmış)
* Evaluation (Test verisinde)

Bu örnek için mnist datasını kullanalım: 

In [4]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1,784).astype("float32")/255
x_test = x_test.reshape(-1,784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# son 10000 data validation için ayrılıyor. 
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [5]:
# Eğitim için gerekli konfigurasyonları yapalım. 
model.compile(optimizer=keras.optimizers.RMSprop(),
             loss = keras.losses.SparseCategoricalCrossentropy(),
             metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [6]:
# fit metodunu çağırarak model eğitimini başlatacağız. 
# Eğitimi datayı batchlere bölerek yapacağız. Batchler halinde tüm verisetinde gezerek belirtilen epoch kadar eğitim yapılacak. 
print("Modeli eğitim verisi üzerinde çalıştıralım")
history = model.fit(x_train,y_train,batch_size=64,epochs=2,validation_data = (x_val,y_val))

Modeli eğitim verisi üzerinde çalıştıralım
Epoch 1/2
782/782 [==============================] - 5s 4ms/step - loss: 0.3405 - sparse_categorical_accuracy: 0.9024 - val_loss: 0.2220 - val_sparse_categorical_accuracy: 0.9342
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.1624 - sparse_categorical_accuracy: 0.9520 - val_loss: 0.1748 - val_sparse_categorical_accuracy: 0.9488


In [7]:
# Fit metodundan bir history objesi döndürdük. 
history.history

{'loss': [0.3405407667160034, 0.16236719489097595],
 'sparse_categorical_accuracy': [0.9024199843406677, 0.9520400166511536],
 'val_loss': [0.22197943925857544, 0.17477765679359436],
 'val_sparse_categorical_accuracy': [0.9341999888420105, 0.9488000273704529]}

In [8]:
# evaluate ile modeli değerlendirelim 
print("Test datası ile evaluate etme: ")
results = model.evaluate(x_test,y_test,batch_size=128)
print("Test loss ve test acc değerleri : ",results)

# Prediction için : 
print("Prediction")
prediction = model.predict(x_test[0].reshape(1,-1))
print(prediction)

Test datası ile evaluate etme: 
79/79 [==============================] - 0s 2ms/step - loss: 0.1782 - sparse_categorical_accuracy: 0.9473
Test loss ve test acc değerleri :  [0.17820633947849274, 0.9473000168800354]
Prediction
[[1.9401762e-06 2.4910689e-08 1.2575711e-04 2.6023012e-04 2.7782872e-09
  5.7926991e-06 1.5849018e-10 9.9960345e-01 7.9646156e-07 1.9024847e-06]]


## Workflodaki adımlara ayrıntılı olarak bakalım
### Compile Metodu, loss-metrics-optimizer
Model eğitimini başlatmak için loss, optimizer ve başarıyı ölçümleyecek metriklere ihtiyaç vardır. Bu parametreleri compile metoduna argument olarak göndeririz.   
Metric parametresi liste şeklinde bir modelin istenilen sayıda metriği olabilir.  
Eğer modelin birden fazla outputu varsa, her output için farklı loss ve metrikler belirleyebiliriz. Her outputun total loss a ne kadar etki edeceğini de ayarlayabiliriz. 

In [9]:
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=1e-3),
             loss = keras.losses.SparseCategoricalCrossentropy(),
             metrics=[keras.metrics.SparseCategoricalAccuracy()])

# ikiside aynı şekilde çalışır 
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [10]:
# DAHA SONRA KULLANMAK İÇİN MODELİ VE COMPILE İŞLEMİ İÇİN FONKSİYONLAR OLUŞTURALIM
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

### Bir çok built-in optimizers, loss, metrik vardır. 
Genellikle kendi loss,metrik, optimizer larını oluşturmamıza gerek kalmaz. Bunların hepsi Keras API de vardır. 
Optimizers: 
* SGD()
* RMSporp()
* Adam()
* etc

Losses: 
* MeanSquaredError()
* KLDivergence()
* CosineSimilarity()
* etc.

Metrics: 
* AUC()
* Precision()
* Recall()
* etc

## Custom Loss 
Eğer custom loss oluşturmak istiyorsak kerasta 2 yöntem vardır.  
y_true ve y_pred değerlerini alan bir fonksiyon tanımlayabiliriz. Aşağıdaki örnekte mean squared error hesaplayan bir fonksiyon bulunuyor. 

In [11]:
def custom_mean_squared_error(y_true,y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred)) 

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(),loss=custom_mean_squared_error)

y_train_one_hot = tf.one_hot(y_train,depth=10)
model.fit(x_train,y_train_one_hot,batch_size=64,epochs=1)


782/782 [==============================] - 2s 2ms/step - loss: 0.0163


Diğer yöntemde ise tf.keras.losses.Loss classını implemente edebiliriz. 
* __init__(self) : call sırasında çağrılacak parametreler için gereklidir
* call(self,y_true,y_pred): target ve predictionları kullanarak loss hesapla

Örneğin, mean squared error kullanmak istiyoruz ama aynı zamanda bir regularization değeri ile modelin 0.5 ten yukarı tahminlerinin etksinin artırmak istiyoruz. Bu sayede overfittingin de önüne geçmiş oluruz. 


In [12]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor
model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 2s 3ms/step - loss: 0.0386


## Custom Metrics
Eğer Keras API de bulunan metrikler dışında bir metrik oluşturmak istersek, kolayca custom metrikler oluşturabiliriz. Bunu için tf.keras.metrics.Metric sınıfından bir sınıf extend ederek aşağıdaki 4 metodu uygulamanız gerekiyor. 
* __init__(self) : metrik için gerekli olan değişkenleri tanımlamak için 
* update_state(self,y_true,y_pred,sample_weight=None): y_true ve y_pred'i kullanarak değişkenleri update eder
* result(self) : Değişkenleri kullanarak son hesaplamayı yapar
* reset_states(self): metrik içerisindeki değerleri tekrar başlatır. 

State_update ve result hesapları ayrı yapılır. Sonuç hesabı belli maliyetli olabilir bu sebeple periyodik olarak yapılmalı.  

Aşağıdaki örnekte CategoricalTruePositives metriğini oluşturalım. Bu metrik ile kaç tane örnek doğru sınıflandırıldığını hesaplıyoruz. 

In [13]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.0)


model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 4s 4ms/step - loss: 0.3398 - categorical_true_positives: 45215.0000
Epoch 2/3
 42/782 [>.............................] - ETA: 2s - loss: 0.1638 - categorical_true_positives: 2561.0000

C:\Users\bagat\anaconda3\envs\tod\lib\site-packages\keras\metrics.py:257: UserWarning: Metric CategoricalTruePositives implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


782/782 [==============================] - 3s 4ms/step - loss: 0.1617 - categorical_true_positives: 47574.0000
Epoch 3/3
782/782 [==============================] - 3s 4ms/step - loss: 0.1155 - categorical_true_positives: 48259.0000
